In [2]:
import pickle
import re
import os
import os.path
import math
import sys
import csv
import glob
import pandas as pd
import seaborn as sns
import numpy as np

from datetime import timedelta, datetime

import matplotlib.pyplot as plt
import seaborn as sns

from enum import Enum    

from sklearn.metrics import confusion_matrix, accuracy_score

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 199)

In [ ]:
### read file

# read csv
df = pd.read_csv("file.csv", header=None, sep="\t") 

#  read csv chunk
num_of_lines = 100000
for chunk in pd.read_csv(csv_url, chunksize = num_of_lines):
    print(chunk.shape)

# read pickle
with open('file.pickle', 'rb') as handle:
    df = pickle.load(handle)
    
# read parquet
pd.read_parquet('file.parquet', engine='fastparquet')

#read parquet/csv folder recursively
def read_folder_into_pd(path, file_type = "parquet"):
    def get_list_files_in_folder(path, file_type = "parquet"):
        file_paths =  glob.glob(os.path.join(path, ("*." + file_type)))
        if len(file_paths) > 0: # one level - do nothing
            pass
        else:
            sub_folder_paths =  glob.glob(os.path.join(path, "*"))
            for p in sub_folder_paths:
                file_paths += get_list_files_in_folder(p)
        return file_paths

    file_path_list = get_list_files_in_folder(path, file_type)
    if file_type == "parquet":
        df = pd.concat((pd.read_parquet(f, 'fastparquet') for f in file_path_list))
    elif file_type == "csv":
        df = pd.concat((pd.read_csv(f) for f in file_path_list))
        
    df.reset_index(drop=True, inplace=True)
    return df

In [ ]:
### write file

# write a list to txt file
with open('your_file.txt', 'w') as f:
    for item in my_list:
        f.write("%s\n" % item)


# write csv from df
df.to_csv("filename.csv", sep=',', index=False)

# write pickle
with open('filename.pickle', 'wb') as handle:
    pickle.dump(df, handle, protocol=pickle.HIGHEST_PROTOCOL)

# append to an existing csv file
with open("accuracy_score.csv", "a") as f:
    w = csv.writer(f, delimiter = ' ')
    list_to_write = [str(best_score), str(best_estimator), 
                  str(X.columns), str(reduce_canNotPredict), str(transform_method)]
    w.writerow(list_to_write)

In [ ]:
### plot

# plot numerical variable
sns_plot = sns.distplot(age)

# plot categorial variable: first, seaborn calculates the num of appearances in each category,then plots, it is equivalent to group_ip.value_counts().plot.bar()
sns_plot = sns.countplot(group_ip)

# other plots
cur_predict_and_label.percent.plot()
cur_predict_and_label.percent.plot.bar()


## options
# rotate 90 degrees
sns_plot.set_xticklabels(g.get_xticklabels(), rotation=90) 

# size
plt.figure(figsize = (60,10)) 

# save to pdf
sns_plot.get_figure().savefig("output.png")

print(age.describe(percentiles=[i* (1/20) for i in range(20)] ))
print("skew : ",age_none_zero.skew())
print("kurt : ", age_none_zero.kurt())

In [ ]:
### matrix

# confusion matrix
cnf_matrix = confusion_matrix(current_prediction, df.label_city_id)
pd.crosstab(df.label_city_id, current_prediction, rownames=['Actual Species'], colnames=['Predicted Species'])

# heat map
def plot_heatmap(col_names, calc_func, df, size = (24,8)):
    
    plt.figure(figsize = size) 
    cor_cols = col_names
    cor_table = np.zeros((len(cor_cols), len(cor_cols)))

    for x, c1 in enumerate(cor_cols):
        for y, c2 in enumerate(cor_cols):
            cor_table[x][y] = calc_func(df[c1], df[c2])

    sns.set(font_scale = 1.25)
    hm = sns.heatmap(cor_table, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cor_cols, xticklabels=cor_cols)
    plt.show()
    
def plot_confusion_matrix_ver2(df, label_col, pred_col, map_label_to_name, fig_size = (10,7)):
    """
    df: data frame
    label_col: label column name
    pred_col: predict column name
    map_label_to_name: map numerical labels to string names. Let it blank if no need to set.
                    example: map_label_to_name = {2: "ORANGE", 1:"APPLE", 0:"DURIAN"}
    fig_size: the size of confusion matrix, default: (10,7)
    """

    df_confusion = pd.crosstab(df[label_col], df[pred_col], rownames=['Actual'], colnames=['Predicted'], margins=True)
    
    if map_label_to_name is None:
        cols = np.unique(df[label_col])
    else:
        cols = [map_label_to_name[c] for c in np.unique(df[label_col])]

    plt.figure(figsize = fig_size)
    sns.set(font_scale=1.0) #for label size
    f = sns.heatmap(df_confusion, annot=True,annot_kws={"size": 15}, fmt='g', cmap="YlGnBu", xticklabels=cols, yticklabels=cols)# font size
    return f

In [ ]:
## rename

path = '../path/'
i = 0
for filename in os.listdir(path):
    os.rename(os.path.join(path,filename), os.path.join(path, filename.replace("old_name", "new_name")))
    i = i +1
for filename in os.listdir(path):
    print(filename)

In [ ]:
## split a col to 3 cols:
top_3 = df['top_3_city'].map(lambda x: re.sub('\[|\]', "", str(x)))  #remove [] by "" in x
df[['city0', 'city1', 'city2']] = top_3.str.split(',', n=2, expand=True)



## create new one:
# approach 1: (faster approach 2)
%%timeit
a,b,c = [], [], []
for row in df2.itertuples(index=True, name='Pandas'):
    t1, t2, t3 = distance(row.long, row.lat)
    a.append(t1)
    b.append(t2)
    c.append(t3)
df2['a'] = a
df2['b'] = b
df2['c'] = c

#approach 2:
temp = df.apply(convert, axis=1)
df[["_c", "_d", "_w"]] = temp.str.split("|", n=2, expand=True)


In [ ]:
### convert categorical variable to numerical one

In [ ]:
### feature selection

In [ ]:
### tuning parameters

In [ ]:
### plot learning curve

In [ ]:
# check size of object
import sys
for i in dir():
    print (i, sys.getsizeof(eval(i)) )

df.info(memory_usage="deep")  : more readable

sys.getsizeof(df)
df.memory_usage(index=True, deep=True).sum()

In [ ]:
#import from other folder
sys.path.append('/path/to/application/app/importedfolder')